In [6]:
# necessary libraries for pre-processing
import utils
import pandas as pd
import numpy as np
import os

# Predictions

### For a first simpler approach, we will only use the 'loan' table

In [7]:
# Reading the loan train & loan test table
loan_df = utils.read_csv_to_df('ficheiros_competicao/loan_train.csv', delimiter=';')

In [8]:
# We should now start 'cleansing' the data here

In [9]:
# Good tutorial for feature engineering:
# https://medium.com/datadriveninvestor/a-simple-guide-to-creating-predictive-models-in-python-part-1-8e3ddc3d7008

In [10]:
# Outputting the resultant table to a final csv
utils.write_df_to_csv(loan_df, 'dataset', 'preprocessed_data.csv')

WindowsError: [Error 183] Impossível criar um ficheiro quando esse ficheiro já existe: 'dataset'